In [1]:
%matplotlib qt
from fenics import *
from mshr import *
from math import sin, cos, pi

a = 1.0   # inner radius of iron cylinder
b = 1.2   # outer radius of iron cylinder
c_1 = 0.8 # radius for inner circle of copper wires
c_2 = 1.4 # radius for outer circle of copper wires
r = 0.1   # radius of copper wires
R = 5.0   # radius of domain
n = 10    # number of windings


In [2]:
# Define geometry for background
domain = Circle(Point(0, 0), R)

# Define geometry for iron cylinder
cylinder = Circle(Point(0, 0), b) - Circle(Point(0, 0), a)

# Define geometry for wires (N = North (up), S = South (down))
angles_N = [i*2*pi/n for i in range(n)]
angles_S = [(i + 0.5)*2*pi/n for i in range(n)]
wires_N = [Circle(Point(c_1*cos(v), c_1*sin(v)), r) for v in angles_N]
wires_S = [Circle(Point(c_2*cos(v), c_2*sin(v)), r) for v in angles_S]


In [3]:
# Set subdomain for iron cylinder
domain.set_subdomain(1, cylinder)

# Set subdomains for wires
for (i, wire) in enumerate(wires_N):
    domain.set_subdomain(2 + i, wire)
for (i, wire) in enumerate(wires_S):
    domain.set_subdomain(2 + n + i, wire)

In [4]:
mesh = generate_mesh(domain, 32)

In [5]:
#How to display
plot(mesh)

In [6]:
markers = MeshFunction('size_t', mesh, 2, mesh.domains())


In [7]:
print(type (markers))
plot(markers)

<class 'dolfin.cpp.mesh.MeshFunctionSizet'>


In [8]:
dx = Measure('dx', domain=mesh, subdomain_data=markers)

In [9]:

# Define function space
V = FunctionSpace(mesh, 'P', 1)

# Define boundary condition
bc = DirichletBC(V, Constant(0), 'on_boundary')

In [10]:
#import sys
#print(sys.getrecursionlimit()) #was 3000
#sys.setrecursionlimit(8000)

In [11]:
# Define magnetic permeability
class Permeability(UserExpression): # UserExpression instead of Expression
    def __init__(self, markers, **kwargs):
        super().__init__(**kwargs) # This part is new!
        self.markers = markers
    def eval_cell(self, values, x, cell):
        if self.markers[cell.index] == 0:
            values[0] = 4*pi*1e-7 # vacuum
        elif self.markers[cell.index] == 1:
            values[0] = 1e-5      # iron (should really be 6.3e-3)
        else:
            values[0] = 1.26e-6   # copper

mu = Permeability(markers, degree=1)

In [13]:
J_N = Constant(1.0)
J_S = Constant(-1.0)
A_z = TrialFunction(V)
v = TestFunction(V)
a = (1 / mu)*dot(grad(A_z), grad(v))*dx
L_N = sum(J_N*v*dx(i) for i in range(2, 2 + n))
L_S = sum(J_S*v*dx(i) for i in range(2 + n, 2 + 2*n))
L = L_N + L_S


In [14]:
W = VectorFunctionSpace(mesh, 'P', 1)
B = project(as_vector((A_z.dx(1), -A_z.dx(0))), W)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to assemble system.
*** Reason:  expected a linear form for L.
*** Where:   This error was encountered inside SystemAssembler.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.2.0.dev0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [25]:
from fenics import *
from mshr import *
from math import sin, cos, pi

a = 1.0   # inner radius of iron cylinder
b = 1.2   # outer radius of iron cylinder
c_1 = 0.8 # radius for inner circle of copper wires
c_2 = 1.4 # radius for outer circle of copper wires
r = 0.1   # radius of copper wires
R = 5.0   # radius of domain
n = 10    # number of windings

# Define geometry for background
domain = Circle(Point(0, 0), R)

# Define geometry for iron cylinder
cylinder = Circle(Point(0, 0), b) - Circle(Point(0, 0), a)

# Define geometry for wires (N = North (up), S = South (down))
angles_N = [i*2*pi/n for i in range(n)]
angles_S = [(i + 0.5)*2*pi/n for i in range(n)]
wires_N = [Circle(Point(c_1*cos(v), c_1*sin(v)), r) for v in angles_N]
wires_S = [Circle(Point(c_2*cos(v), c_2*sin(v)), r) for v in angles_S]

# Set subdomain for iron cylinder
domain.set_subdomain(1, cylinder)

# Set subdomains for wires
for (i, wire) in enumerate(wires_N):
    domain.set_subdomain(2 + i, wire)
for (i, wire) in enumerate(wires_S):
    domain.set_subdomain(2 + n + i, wire)

# Create mesh
mesh = generate_mesh(domain, 128)

# Define function space
V = FunctionSpace(mesh, 'P', 1)

# Define boundary condition
bc = DirichletBC(V, Constant(0), 'on_boundary')

# Define subdomain markers and integration measure
markers = MeshFunction('size_t', mesh, 2, mesh.domains())
dx = Measure('dx', domain=mesh, subdomain_data=markers)

# Define current densities
J_N = Constant(1.0)
J_S = Constant(-1.0)


# Define magnetic permeability
class Permeability(UserExpression): # UserExpression instead of Expression
    def __init__(self, markers, **kwargs):
        super().__init__(**kwargs) # This part is new!
        self.markers = markers
    def eval_cell(self, values, x, cell):
        if self.markers[cell.index] == 0:
            values[0] = 4*pi*1e-7 # vacuum
        elif self.markers[cell.index] == 1:
            values[0] = 1e-5      # iron (should really be 6.3e-3)
        else:
            values[0] = 1.26e-6   # copper

mu = Permeability(markers, degree=1)

mu = Permeability(markers, degree=1)

# Define variational problem
A_z = TrialFunction(V)
v = TestFunction(V)
a = (1 / mu)*dot(grad(A_z), grad(v))*dx
L_N = sum(J_N*v*dx(i) for i in range(2, 2 + n))
L_S = sum(J_S*v*dx(i) for i in range(2 + n, 2 + 2*n))
L = L_N + L_S

print(L_N)
print(L_S)
print(L)
# Solve variational problem
A_z = Function(V)
solve(a == L, A_z, bc)

# Compute magnetic field (B = curl A)
W = VectorFunctionSpace(mesh, 'P', 1)
B = project(as_vector((A_z.dx(1), -A_z.dx(0))), W)

# Plot solution
plot(A_z)
plot(B)

# Save solution to file
vtkfile_A_z = File('magnetostatics/potential.pvd')
vtkfile_B = File('magnetostatics/field.pvd')
vtkfile_A_z << A_z
vtkfile_B << B

# Hold plot
#interactive()

{ v_0 * f_3203 } * dx(<Mesh #2821>[2], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[3], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[4], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[5], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[6], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[7], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[8], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[9], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[10], {})
  +  { v_0 * f_3203 } * dx(<Mesh #2821>[11], {})
{ v_0 * f_3204 } * dx(<Mesh #2821>[12], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[13], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[14], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[15], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[16], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[17], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[18], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[19], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[20], {})
  +  { v_0 * f_3204 } * dx(<Mesh #2821>[21], {})
{ v_0 * f_3203 } * dx(<Mesh #2821>[2],

In [17]:
plot(B)

In [18]:
plot(A_z)

In [21]:
"""
First trial in FEniCS to implement the Maxwell eigenvalue problem in 2D/3D using Nedelec elements
(here compared with the Laplace eigenvalue problem using Lagrange elements)
-Nabla x Nabla x u = k^2 u   on the domain
			 n x u = 0 	     on the boundary
     
"""

from dolfin import *


#----------- preliminary settings and checks -----------------------------------------------------------------------------------------------------------------


# Test for PETSc and SLEPc
if not has_linear_algebra_backend("PETSc"):
    print ("DOLFIN has not been configured with PETSc. Exiting.")
    exit()

if not has_slepc():
    print ("DOLFIN has not been configured with SLEPc. Exiting.")
    exit()


# Get user input about which problem shall be solved
prob = raw_input("Please specify if you want to solve the Laplace or the Maxwell eigenproblem. ")
prob = prob.capitalize()
allowed_problems = ["Maxwell", "Laplace"]
if prob in allowed_problems:
    print ("The ", prob, " eigenvalue problem will be solved.")
else:
    print ("That was not a valid choice.")
    exit()
if prob == "Maxwell":
    ind = 1
else: 
    ind = 0
    
    
# Get user input about how many eigenvalues to compute
n_eigs = raw_input("Please specify the number of eigenvalues you want computed. ")
n = int(n_eigs)
if n > 100:
    print ("Sorry, that are too many.")
    exit()
elif n < 1:
    print ("Sorry, that are too few.")
    exit()
else:
    print ("The first ", n, " eigenvalues and eigenfunctions will be computed.")


# Get user input about how many grid points to use
grid_p = raw_input("Please specify the number of grid points to be used. ")
p = int(grid_p)
if p > 100:
    print ("Sorry, that are too many.")
    exit()
elif p < 1:
    print ("Sorry, that are too few.")
    exit()
    
    
#----------- actual calculations -----------------------------------------------------------------------------------------------------------------


# Define mesh, function space
#mesh = UnitCubeMesh(p,p,p)
mesh = UnitSquareMesh(p,p)
#mesh = CircleMesh(Point(0,0,0), 1, 0.05)

# Maxwell or Laplace function space
if ind > 0:
    V = FunctionSpace(mesh, "N1curl", 1)
else:
    V = FunctionSpace(mesh, "Lagrange", 1)


# Define basis and bilinear form
u = TrialFunction(V)
v = TestFunction(V)

# Maxwell or Laplace bilinear form
if ind > 0:
    a = dot(curl(u), curl(v))*dx
else:
    a = dot(grad(u), grad(v))*dx


# Assemble stiffness form
A = PETScMatrix()
assemble(a, tensor=A)

# Create eigensolver
eigensolver = SLEPcEigenSolver(A)

# Compute n eigenvalues of A x = \lambda x
print ("Computing the first", n, "eigenvalues. This can take a minute.")
eigensolver.solve(n)


#----------- vizualization -----------------------------------------------------------------------------------------------------------------


for i in range(0,n): 
    # Extract ith eigenpair
    r, c, rx, cx = eigensolver.get_eigenpair(i)

    print(i, ". eigenvalue: ", r)

    # Initialize function and assign eigenvector
    u = Function(V)
    u.vector()[:] = rx

    # Plot eigenfunction
    plot(u)

interactive()

NameError: name 'raw_input' is not defined